In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>


In [2]:
from openai import OpenAI
import os
from langchain.prompts import PromptTemplate
from langchain.llms import openai
from langchain.chains import LLMChain

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

llm = openai.OpenAI(model="text-davinci-003", max_tokens=2048, temperature=0.5)
multiply_prompt = PromptTemplate(template="Please calculate what {question} is?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run({"question": "496 multiplied by 537"})
print("OpenAI API says the answer is:", answer)

python_answer = 496 * 537
print("Python says the answer is:", python_answer)


OpenAI API says the answer is: 

264,672
Python says the answer is: 266352


In [5]:
multiply_by_python_prompt = PromptTemplate(template="Please write a Python code to calculate {question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = math_chain.run({"question": "496 multiplied by 537"})
print(answer)




answer = 496 * 537
print(answer)

Output: 265952


In [5]:
multiply_by_python_prompt = PromptTemplate(template="Please write a Python code to calculate {question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "496 multiplied by 537"})

from langchain.utilities import PythonREPL
python_repl = PythonREPL()
result = python_repl.run(answer_code)
print(result)


Python REPL can execute arbitrary code. Use with caution.


266352



In [6]:
from langchain import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)
result = llm_math.run("Please calculate what 496 multiplied by 537 is?")
print(result)




> Entering new LLMMathChain chain...
Please calculate what 496 multiplied by 537 is?
```text
496 * 537
```
...numexpr.evaluate("496 * 537")...

Answer: 266352
> Finished chain.
Answer: 266352


In [20]:
from langchain.chains import LLMRequestsChain

template = """Between >>> and <<< is the original search result from Google.
Please extract the answer to the question '{query}' from it. If there is no relevant information, say "Not found".
Please use the following format:
Extracted:<answer or "Not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)
requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=openai.OpenAI(temperature=0), prompt=PROMPT))
question = "What is the weather like in Los Angeles today?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
result=requests_chain(inputs)
print(result)
print(result['output'])


{'query': 'What is the weather like in Los Angeles today?', 'url': 'https://www.google.com/search?q=What+is+the+weather+like+in+Los+Angeles+today?', 'output': ' Sunny to partly cloudy. Temperature: 2780°C. Chance of Rain: 0%. Wind Speed: 3 km/h (2 mph).'}
 Sunny to partly cloudy. Temperature: 2780°C. Chance of Rain: 0%. Wind Speed: 3 km/h (2 mph).


In [21]:
import re

def analyze_weather_forecast(text):
    # Regular expression pattern
    weather_pattern = r"^(.*?)\."
    temperature_pattern = r"Temperature: (\d+)°C"
    rain_chance_pattern = r"Chance of Rain: (\d+)%"
    wind_speed_pattern = r"Wind Speed: (\d+ km/h)"

    # Search for matches
    weather = re.search(weather_pattern, text)
    temperature = re.search(temperature_pattern, text)
    rain_chance = re.search(rain_chance_pattern, text)
    wind_speed = re.search(wind_speed_pattern, text)

    # Extract results
    weather = weather.group(1) if weather else "Not available"
    temperature = temperature.group(1) if temperature else "Not available"
    rain_chance = rain_chance.group(1) if rain_chance else "Not available"
    wind_speed = wind_speed.group(1) if wind_speed else "Not available"

    return {
        "Weather": weather,
        "Temperature": temperature,
        "Rain Chance": rain_chance,
        "Wind Speed": wind_speed
    }

# Example.
forecast_text = "Sunny to partly cloudy. Temperature: 2780°C. Chance of Rain: 0%. Wind Speed: 3 km/h (2 mph)."
result = analyze_weather_forecast(forecast_text)
print(result)


{'Weather': 'Sunny to partly cloudy', 'Temperature': '2780', 'Rain Chance': '0', 'Wind Speed': '3 km/h'}


In [22]:
from langchain.chains import TransformChain, SequentialChain

def transform_func(inputs: dict) -> dict:
    text = inputs["output"]
    return {"weather_info" : analyze_weather_forecast(text)}

transformation_chain = TransformChain(input_variables=["output"],
                                      output_variables=["weather_info"], transform=transform_func)

final_chain = SequentialChain(chains=[requests_chain, transformation_chain],
                              input_variables=["query", "url"], output_variables=["weather_info"])
final_result = final_chain.run(inputs)
print(final_result)


{'Weather': ' Sunny to partly cloudy', 'Temperature': '2780', 'Rain Chance': '0', 'Wind Speed': '3 km/h'}


In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import faiss
from langchain.text_splitter import SpacyTextSplitter
from langchain.chains.retrieval_qa.base import VectorDBQA
from langchain.document_loaders import TextLoader

llm = openai.OpenAI(temperature=0)
loader = TextLoader('./data/amazon_faq.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="en_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = faiss.FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)


/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Created a chunk of size 434, which is longer than the specified 256
Created a chunk of size 466, which is longer than the specified 256
/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:251: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [24]:
question = "What is FBA?"
result = faq_chain.run(question)
print(result)




> Entering new VectorDBQA chain...

> Finished chain.
 FBA stands for Fulfillment by Amazon. It is a pay-as-you-go fulfillment service where you pay for storage space and fulfillment of each sale. The cost of shipping is included in your fees, with no extra charge for Amazon Prime FREE Two-Day Shipping and free shipping on eligible orders.


In [25]:
question = "What sells the most in Amazon’s store?"
result = faq_chain.run(question)
print(result)




> Entering new VectorDBQA chain...

> Finished chain.
 Top selling items on any given day may include books, baby items, electronics, kitchen supplies, outdoor gear—the list goes on. Check out Amazon Best Sellers for an hourly update with the most popular products based on sales.
